# Negative Weighted Events

In this notebook, we adapt the negative events-based measure from van den Broucke et al. from 2014 in the paper: "Determining Process Model Precision and Generalization with Weighted Artificial Negative Events" (doi: 10.1109/TKDE.2013.130). <br>
With respect to the actual executions of a process, the measure focuses on negative events, where a negative event represents information about activities that were prevented from taking place in the first place. Since they are rarely recorded in reality, we induce them into the log artificially. <br>
So, we basically induce all elements that weren't fired at this specific position in the event log. For simplicity, the authors assume that all other events, outside the current event itself, are inserted at each position in the trace. Afterwards, we check which of these events could be fired at the current position in the corresponding position or not. If an event can be fired, we increase a counter for allowed generalizations $AG$ by one, and if not, we increase a counter for allowed generalizations $DG$ by one. <br>
The final measure is calculated as follows: <br>
For event log $E$ and process model $M$,
$$ Generalization (L,M) = AG\,/\, (AG+DG).$$
One has to be aware, that this is not the complete logic of the paper, which is implemented in a further step by introducing a scoring mechanism. This is done due to the fact that we have to assume the completeness of an event log to induce negative events and this is normally not true in reality, as an event log only represents a subset. The scoring mechanism will allow us to loosen this assumption by only assuming the completeness property on a small window before the execution of the current activity.

In [4]:
import warnings
warnings.filterwarnings('ignore')
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from models.negative_events_measure import negative_events_without_weighting

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 46/46 [00:00<00:00, 46312.53it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 170.00it/s]


0.5063

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 241.53it/s]


0.3073

### Flower Model Petri Net

In [8]:
ots = ["order","item","delivery"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 32/32 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 223.43it/s]


0.965

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [11]:
filename_variant = "../src/data/csv/order_process_variant_log.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [12]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 9409.54it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [13]:
value = negative_events_without_weighting (ocel_variant, variant_ocpn)
value

Check the arcs: 100%|██████████| 378/378 [00:00<00:00, 17582.47it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:02<00:00, 19.65it/s]


0.1375

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [14]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [15]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 40/40 [00:00<00:00, 77065.76it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 838.91it/s]


0.1845

### Happy Path Petri Net

In [16]:
happy_path__ocel = get_happy_path_log(filename)

In [17]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<00:00, 266974.12it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 781.10it/s]


0.1958

### Flower Model Petri Net

In [19]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [20]:
flower_ocpn = create_flower_model(filename,ots)

In [21]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<00:00, 38212.31it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 732.80it/s]


0.8611

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [22]:
filename_variant = "../src/data/csv/p2p_variant_log.csv" 
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [23]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p_variants/p2p_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 20/20 [00:00<00:00, 9618.86it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [24]:
value = negative_events_without_weighting (ocel_variant, variant_ocpn)
value

Check the arcs: 100%|██████████| 760/760 [00:00<00:00, 21810.51it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 80.15it/s]


0.115

# BPI-Challenge 2017 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [25]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [26]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 120/120 [00:00<00:00, 118650.75it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:00<00:00, 524.52it/s]


0.3569

### Happy Path Petri Net

In [27]:
happy_path__ocel = get_happy_path_log(filename)

In [28]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [29]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<00:00, 24363.70it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:00<00:00, 522.93it/s]


0.1077

### Flower Model Petri Net

In [30]:
ots = ["application","offer"]

In [31]:
flower_ocpn = create_flower_model(filename,ots)

In [32]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 56/56 [00:00<00:00, 19040.29it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:04<00:00, 487.77it/s]


0.8827

# Run parallelism for variant models

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import logging
import multiprocessing
def filter_silent_transitions(dic,silent_transitions):
    """
    Function to filter out the silent transitions defined by a list from a given dictionary.
    :param dic: dictionary to be filtered, type: dictionary
    :param silent_transitions: list of silent transitions in an ocel log, type: list
    :return updated_dictionary: filtered dictionary, type: dictionary
    """
    updated_dictionary = {}
    for key, values in dic.items():
        if key not in silent_transitions:
            new_values = [val for val in values if val not in silent_transitions]
            updated_dictionary[key] = new_values
    return updated_dictionary

#recursive implementation of a depth-first search (DFS) algorithm
def dfs(graph, visited, activity, preceding_events):
    """
    Function to perform a depth-first search (DFS) algorithm on the activity graph.
    :param graph: activity graph, type: dictionary
    :param visited: set of already visited nodes, type: set
    :param activity: current activity, type: string
    :param preceding_events: list to store the preceding events, type: list
    """
    #takes as input the activity graph (represented as a dictionary), a set of visited nodes, the current activity, and a list to store the preceding events.
    visited.add(activity)
    for preceding_event in graph[activity]:
        #eighboring activity has not been visited yet, the algorithm visits it by calling the dfs function with the neighboring activity as the current activity.
        if preceding_event not in visited:
            dfs(graph, visited, preceding_event, preceding_events)
    preceding_events.append(activity)


def update_global_variables_without(group, filtered_preceding_events_full, filtered_preceding_events,
                            filtered_succeeding_activities_updated, events, silent_transitions, AG, DG):
    """
    Function to update the global variables AG and DG based on the paper logic.
    :param group: group of the event log that is currently processed, type: pandas dataframe
    :param filtered_preceding_events_full: dictionary that contains all preceding events for each activity, type: dictionary
    :param filtered_preceding_events: dictionary that contains all directly preceding events for each activity, type: dictionary
    :param filtered_succeeding_activities_updated: dictionary that contains all succeeding activities for each activity, type: dictionary
    :param events: list of all activities in the process model, type: list
    :param silent_transitions: list of all silent transitions in the process model, type: list
    :param AG: global variable for allowed generalizations, type: int
    :param DG: global variable for disallowed generalizations, type: int
    :return: updated values for AG and DG, type: int
    """
    # Iterate over each row in the group
    # list for all the activities that are enabled, starting from all activities that do not have any preceding activity
    enabled = [key for key, value in filtered_preceding_events_full.items() if not value]
    # initialise a list of already executed activities in this trace
    trace = []
    # iterate through each case/process execution
    for index, row in group.iterrows():
        # Get the current negative events based on the current activity to be executed
        negative_activities = [x for x in events if x != row['event_activity']]
        # it may happen that an activity is not present in the model but nevertheless executed in the log
        if row['event_activity'] in enabled:
            # check which elements in the negative activity list are enabled outside of the current activity
            enabled.remove(row['event_activity'])
        # get all the negative events that can not be executed in the process model at the moment
        disallowed = [value for value in negative_activities if value not in enabled]
        # add activity that has been executed to trace
        trace.append(row['event_activity'])
        # update the values of allowed and disallowed generalizations based on the paper logic
        AG = AG + len(enabled)
        DG = DG + len(disallowed)

        # may happen that activities in the log are not in the process model
        if row['event_activity'] in filtered_succeeding_activities_updated:
            # get all possible new enabled activities
            possible_enabled = filtered_succeeding_activities_updated[row['event_activity']]
            # check if each activity has more than one directly preceding state
            for i in range(len(possible_enabled)):
                # check if an event has two or more activities that need to be executed before the event can take place, if not add events to enabled
                if len(filtered_preceding_events[possible_enabled[i]]) < 2:
                    enabled.append(possible_enabled[i])
                # if all succeeding events equal all preceding events, we have a flower model and almost everything is enabled all the time
                elif filtered_preceding_events[possible_enabled[i]] == filtered_succeeding_activities_updated[
                    possible_enabled[i]]:
                    enabled.append(possible_enabled[i])
                else:
                    # if yes, check if all the needed activities have already been performed in this trace
                    if all(elem in trace for elem in filtered_preceding_events[possible_enabled[i]]):
                        enabled.append(possible_enabled[i])
        # extend the list with all elements that do not have any preceding activity and are therefore enabled anyways in our process model
        enabled.extend([key for key, value in filtered_preceding_events_full.items() if not value])
        # delete all duplicates from the enabled list
        enabled = list(set(enabled))
    return AG, DG

# Define the function that will be executed in parallel
def process_group_without(args):
    """
    Function to process a group of the event log in parallel
    :param args: set of variables for the measure calculation (see original function)
    :return: updated values for AG and DG, type: int
    """
    group_key, df_group, filtered_preceding_events_full, filtered_preceding_events, \
    filtered_succeeding_activities_updated, events, silent_transitions, AG, DG = args
    AG, DG = update_global_variables_without(df_group, filtered_preceding_events_full, filtered_preceding_events,
                                     filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
    return AG, DG

def negative_events_without_weighting_parallel(ocel, ocpn):
    """
    Function to calculate the variables for negative events measure paralle calculation without weighting
    :param ocel: object-centric event log for which the measure should be calculated, type: ocel-log
    :param ocpn: corresponding object-centric petri-net, type: object-centric petri-net
    :return: set of variables for the measure calculation (see original function)
    """
    # since the process execution mappings have lists of length one,
    # we create another dictionary that only contains the  value inside the list to be able to derive the case
    mapping_dict = {key: ocel.process_execution_mappings[key][0] for key in ocel.process_execution_mappings}
    # we generate a new column in the class (log) that contains the process execution (case) number via the generated dictionary
    ocel.log.log['event_execution'] = ocel.log.log.index.map(mapping_dict)
    # generate a list of unique events in the event log
    events = np.unique(ocel.log.log.event_activity)
    # dictionary to store each activity as key and a list of its prior states/places as value
    targets = {}
    # dictionary to store each activity as key and a list of its following states/places as value
    sources = {}
    for arc in tqdm(ocpn.arcs, desc="Check the arcs"):
        # for each arc, check if our target is a valid transition
        if arc.target in ocpn.transitions:
            # load all the prior places of a valid transition into a dictionary, where the key is the transition and the value
            # a list of all directly prior places
            if arc.target.name in targets:
                targets[arc.target.name].append(arc.source.name)
            else:
                targets[arc.target.name] = [arc.source.name]
        if arc.source in ocpn.transitions:
            # load all the following places of a valid transition into a dictionary, where the key is the transition and the value
            # a list of all directly following places
            if arc.source.name in sources:
                sources[arc.source.name].append(arc.target.name)
            else:
                sources[arc.source.name] = [arc.target.name]
    # generate an empty dictionary to store the directly preceding transition of an activity
    preceding_activities = {}
    # use the key and value of targets and source to generate the dictionary
    for target_key, target_value in targets.items():
        preceding_activities[target_key] = []
        for source_key, source_value in sources.items():
            for element in target_value:
                if element in source_value:
                    preceding_activities[target_key].append(source_key)
                    break
    # generate an empty dictionary to store the directly succeeding transition of an activity
    succeeding_activities = {}
    for source_key, source_value in sources.items():
        succeeding_activities[source_key] = []
        for target_key, target_value in targets.items():
            for element in source_value:
                if element in target_value:
                    succeeding_activities[source_key].append(target_key)
                    break
    # store the name of all silent transitions in the log
    silent_transitions = [x.name for x in ocpn.transitions if x.silent]
    # replace the silent transitions in the succeeding activities dictionary by creating a new dictionary to store the modified values
    succeeding_activities_updated = {}
    # Iterate through the dictionary
    for key, values in succeeding_activities.items():
        # Create a list to store the modified values for this key
        new_values = []
        # Iterate through the values of each key
        for i in range(len(values)):
            # Check if the value is in the list of silent transitions
            if values[i] in silent_transitions:
                # Replace the value with the corresponding value from the dictionary
                new_values.extend(succeeding_activities[values[i]])
            else:
                # If the value is not in the list of silent transitions, add it to the new list
                new_values.append(values[i])
        # Add the modified values to the new dictionary
        succeeding_activities_updated[key] = new_values
    # create an empty dictionary to store all the preceding activities of an activity
    preceding_events_dict = {}
    # use a depth-first search (DFS) algorithm to traverse the activity graph and
    # create a list of all preceding events for each activity in the dictionary for directly preceding activities
    for activity in preceding_activities:
        # empty set for all the visited activities
        visited = set()
        # list for all currently preceding events
        preceding_events = []
        dfs(preceding_activities, visited, activity, preceding_events)
        # we need to remove the last element from the list because it corresponds to the activity itself
        preceding_events_dict[activity] = preceding_events[:-1][::-1]
    # delete all possible silent transitions from preceding_events_dict (dict where all direct preceding events are stored)
    filtered_preceding_events_full = filter_silent_transitions(preceding_events_dict, silent_transitions)
    # delete all possible silent transitions from filtered_preceding_events (dict where only direct preceding events are stored)
    filtered_preceding_events = filter_silent_transitions(preceding_activities, silent_transitions)
    # delete all possible silent transitions from succeeding_activities_updated (dict where only direct preceding events are stored)
    filtered_succeeding_activities_updated = filter_silent_transitions(succeeding_activities_updated,
                                                                       silent_transitions)
    # generate a grouped df such that we can iterate through the log case by case (sort by timestamp to ensure the correct process sequence)
    grouped_df = ocel.log.log.sort_values('event_timestamp').groupby('event_execution')

    return grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions



### Variant Model Petri Net

We import the pickle file for the variant model of the bpi challenge that we generated in the process models notebook.

In [2]:
variant_ocel = pd.read_pickle('/pfs/data5/home/ma/ma_ma/ma_nsabel/Generalization_in_Object_Centric_Process_Mining/src/data/csv/bpi_variant.pickle')


In [3]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [5]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(variant_ocel, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(20)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

  1%|          | 219/31509 [42:54<102:11:14, 11.76s/it]Process ForkPoolWorker-4:
Process ForkPoolWorker-20:
Process ForkPoolWorker-9:
Process ForkPoolWorker-1:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-17:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-18:
Process ForkPoolWorker-6:
Process ForkPoolWorker-11:
Process ForkPoolWorker-19:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib64/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File

# DS3 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/DS3.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting(ocel,ocpn)
value

Check the arcs: 100%|██████████| 130/130 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:11<00:00, 405.91it/s] 


0.3701

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting(ocel,happy_path_ocpn)
value

Check the arcs: 100%|██████████| 8/8 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:12<00:00, 395.43it/s] 


0.2073

### Flower Model Petri Net

In [8]:
ots = ["incident","customer"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting(ocel,flower_ocpn)
value

Check the arcs: 100%|██████████| 20/20 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:12<00:00, 373.28it/s] 


0.9588

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [ ]:
variant_ocel = pd.read_pickle('/pfs/data5/home/ma/ma_ma/ma_nsabel/Generalization_in_Object_Centric_Process_Mining/src/data/csv/ds3_variant.pickle')

In [4]:
with open("../src/data/csv/DS3_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [5]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(variant_ocel, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(20)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

Check the arcs:  67%|██████▋   | 98030/146804 [47:10<29:01, 28.00it/s]  

# DS4 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/DS4.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting(ocel,ocpn)
value

Check the arcs: 100%|██████████| 364/364 [00:00<00:00, 168271.43it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [05:39<00:00, 42.69it/s]


0.3604

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting(ocel,happy_path_ocpn)
value

Check the arcs: 100%|██████████| 70/70 [00:00<00:00, 72512.05it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [05:07<00:00, 47.22it/s]


0.0842

### Flower Model Petri Net

In [8]:
ots = ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting(ocel,flower_ocpn)
value

Check the arcs: 100%|██████████| 162/162 [00:00<00:00, 23182.44it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [18:09<00:00, 13.31it/s]


0.6885

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [ ]:
variant_ocel = pd.read_pickle('/pfs/data5/home/ma/ma_ma/ma_nsabel/Generalization_in_Object_Centric_Process_Mining/src/data/csv/DS4_variant.pickle')

In [ ]:
with open("../src/data/csv/DS4_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [ ]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(variant_ocel, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(20)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))